In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%load_ext memory_profiler

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from arc import Rubidium85

In [5]:
import rydiqule as rq

### Run the memory line profiler to try to get accurate estimates of how much memory we actually require to solve a system.

In [6]:
kb = 1.38e-23
mass = 1.41e-25
temp = 350
vP = np.sqrt(2*kb*temp/mass) # most probable speed
print(f'Most probable speed {vP:.2f} m/s')

basis_size = 4
gam = np.zeros((basis_size,basis_size),dtype=np.float64)
gam[1,0] = 5.75
gam[2,1] = 3.5
gam[3,2] = 10e-3

lambP = 794.979e-3
lambD = 1323.876e-3
lambC = 745.619e-3

kp = 2*np.pi/lambP*np.array([1,0,0])
kd = 2*np.pi/lambD*np.array([1,0,0])
kc = 2*np.pi/lambC*np.array([-1,0,0])

num_dets = 101
det_width = 25

detunings = np.linspace(-det_width,det_width,num_dets)

probe = {'states':(0,1), 'kvec':vP*kp, 'detuning':2*np.pi*detunings, 'rabi_frequency':2*np.pi*10}
dress = {'states':(1,2), 'kvec':vP*kd, 'detuning':0, 'rabi_frequency':2*np.pi*100}
couple = {'states':(2,3), 'kvec':vP*kc, 'detuning':2*np.pi*detunings, 'rabi_frequency':2*np.pi*10}

CL = rq.Sensor(basis_size)
CL.add_couplings(probe,dress,couple)
CL.set_gamma_matrix(2*np.pi*gam)

Most probable speed 261.75 m/s


In [7]:
n =4
stack = (101, 101)
doppler_axis_shape = (561,561)
rq.get_slice_num(n, stack, doppler_axis_shape, sum_doppler=True, weight_doppler=True, n_slices=1, debug=True)

Total available memory: 236.33 GiB
Min Req memory to solve: 0.60152 GiB
Req memory per EOM: 0.59795 GiB
Req memory for full solve: 6099.7 GiB
	Mandatory memory use: 0.0035722 GiB
	Memory use for all EOMs: 6099.7 GiB
	Full output solution size: 0.0011401 GiB
Available memory for sliced solves: 236.32 GiB
Number of stack slices to be used: 26


(26, (101, 101, 15))

In [8]:
n = 8
stack = (101, 101)
doppler_axis_shape = (561,561)
rq.get_slice_num(n, stack, doppler_axis_shape, sum_doppler=False, weight_doppler=False, n_slices=1, debug=True)

Total available memory: 236.33 GiB
Min Req memory to solve: 9.4644 GiB
Req memory per EOM: 9.4546 GiB
Req memory for full solve: 96447 GiB
	Mandatory memory use: 0.014517 GiB
	Memory use for all EOMs: 96447 GiB
	Full output solution size: 1507 GiB
Available memory for sliced solves: 236.31 GiB


MemoryError: Setting 'sum_doppler=False' if full equations of motion do not fit in memory is unsupported.

In [9]:
n = 4
stack = (101, 101)
doppler_axis_shape = (561, 561, 561)
rq.get_slice_num(n, stack, doppler_axis_shape, sum_doppler=True, weight_doppler=True, n_slices=1, debug=True)

Total available memory: 236.31 GiB
Min Req memory to solve: 335.45 GiB
Req memory per EOM: 335.44 GiB
Req memory for full solve: 3.4219e+06 GiB
	Mandatory memory use: 0.0035722 GiB
	Memory use for all EOMs: 3.4219e+06 GiB
	Full output solution size: 0.0011401 GiB
Available memory for sliced solves: 236.3 GiB


MemoryError: System is too large to solve. Need at least 335.4466984719038 GiB

### Memory profiling of steady-state solve ###

Note that the increment will not report correctly if a function on that line is being profiled separately.
You can see the increment by looking at the change in total mem usage between lines in that case.

In [11]:
from rydiqule.solvers import _solve_hamiltonian_stack

In [12]:
%mprun -c -f rq.solve_steady_state -f _solve_hamiltonian_stack rq.solve_steady_state(CL,doppler=True,sum_doppler=True,weight_doppler=True)

Filename: C:\Users\naqsL\src\rydiqule\src\rydiqule\solvers.py

Line #    Mem usage    Increment  Occurences   Line Contents
    15    209.9 MiB    209.9 MiB           1   def solve_steady_state(
    16                                                 sensor: Sensor, doppler: bool = False, doppler_mesh_method: Optional[MeshMethod] = None,
    17                                                 sum_doppler: bool = True, weight_doppler: bool = True,
    18                                                 n_slices: int = 1, **kwargs,
    19                                                 ) -> Solution:
    20                                             """
    21                                             Finds the steady state solution for a system characterized by sensor. 
    22                                             
    23                                             If insuffucent system
    24                                             memory is available to solve the system all 

In [13]:
from rydiqule.sensor_utils import make_real

In [14]:
%mprun -f rq.generate_eom -f make_real eoms, const = rq.generate_eom(CL.get_hamiltonian(), CL.decoherence_matrix())

Filename: C:\Users\naqsL\src\rydiqule\src\rydiqule\sensor_utils.py

Line #    Mem usage    Increment  Occurences   Line Contents
    28    217.5 MiB    217.5 MiB           1   def generate_eom(hamiltonian: np.ndarray, gamma_matrix: np.ndarray,
    29                                                          remove_ground_state: bool = True,
    30                                                          real_eom: bool = True) -> Tuple[np.ndarray, np.ndarray]:
    31                                             """
    32                                             Create the optical bloch equations for a hamiltonian and decoherence matrix
    33                                             using the Lindblad master equation.
    34                                         
    35                                             Parameters
    36                                             ----------
    37                                             hamiltonian : numpy.ndarray
    38           

In [15]:
dop_classes = rq.doppler_classes()

In [16]:
%memit vels, vols = rq.doppler_mesh(dop_classes, CL.spatial_dim())

peak memory: 232.65 MiB, increment: 0.07 MiB


In [17]:
%memit rq.solve_steady_state(CL,doppler=True,sum_doppler=True)

peak memory: 11371.10 MiB, increment: 11138.45 MiB


In [18]:
%mprun -f rq.get_doppler_equations rq.get_doppler_equations(eoms, CL.get_doppler_shifts(), vels)

Filename: C:\Users\naqsL\src\rydiqule\src\rydiqule\doppler_utils.py

Line #    Mem usage    Increment  Occurences   Line Contents
    40    233.8 MiB    233.8 MiB           1   def get_doppler_equations(base_eoms: np.ndarray,
    41                                                                   doppler_hamiltonians: np.ndarray, Vs: np.ndarray) -> np.ndarray:
    42                                             """
    43                                             Returns the equations for each slice of the doppler profile.
    44                                         
    45                                             A new axes corresponding to these slices are appended to the beginning.
    46                                             For example, if equations are of shape `(m,m)`
    47                                             and there are `n_doppler` doppler values being sampled,
    48                                             the return will be of shape `(n_doppler, m